# 1. Install libraries and import modules

In [1]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Setup MediaPipe Holistic Model

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [8]:
def extract_hand_keypoints(results):
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return rh

In [25]:
# create a path for colle
DATA_PATH = os.path.join('data') 

# Actions that we try to detect
actions = np.array(['rotate','switch', 'grab', 'input'])

# 30 videos worth of data
num_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# 4. Setup Folders for Data Collection

Don't run unless it's the first time setup

In [17]:
# Run this only for when setting up folders
for action in actions: 
    for sequence in range(num_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [18]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.6, min_tracking_confidence=0.6) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(30):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    # Collection starts
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    # Display which frame it is for which action
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    # wait for 3s to prepare for next sequence collection
                    cv2.waitKey(3000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_hand_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('x'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [20]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [23]:
label_dict = {label:num for num, label in enumerate(actions)}

In [26]:
sequences, labels = [], []
for action in actions:
    for sequence in range(num_sequences):
        window = []
        for frame_num in range(sequence_length):
            # load the samples stored as numpy array files 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_dict[action])

In [27]:
x = np.array(sequences)

In [28]:
x.shape

(120, 30, 126)

In [29]:
y = to_categorical(labels).astype(int)

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

# 7. Build and Train LSTM

In [43]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

### 7.1 Setting up logs for monitoring training in Tensorboard

To run this, cd into logs/train/ 
Run command: tensorboard --logdir=.
You can then access the dashboard by going to a localhost address

In [44]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

### 7.2 Building Sequential model

In [45]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [46]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [47]:
model.fit(x_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
4/4 [==============================] - 2s 32ms/step - loss: 1.3861 - categorical_accuracy: 0.2544
Epoch 2/500
4/4 [==============================] - 0s 30ms/step - loss: 1.3787 - categorical_accuracy: 0.2632
Epoch 3/500
4/4 [==============================] - 0s 31ms/step - loss: 1.3684 - categorical_accuracy: 0.2719
Epoch 4/500
4/4 [==============================] - 0s 29ms/step - loss: 1.3410 - categorical_accuracy: 0.2807
Epoch 5/500
4/4 [==============================] - 0s 28ms/step - loss: 1.3247 - categorical_accuracy: 0.3246
Epoch 6/500
4/4 [==============================] - 0s 27ms/step - loss: 1.2951 - categorical_accuracy: 0.5088
Epoch 7/500
4/4 [==============================] - 0s 27ms/step - loss: 1.1929 - categorical_accuracy: 0.5263
Epoch 8/500
4/4 [==============================] - 0s 27ms/step - loss: 1.1892 - categorical_accuracy: 0.5263
Epoch 9/500
4/4 [==============================] - 0s 28ms/step - loss: 1.0714 - categorical_accuracy: 0.5614
Epoch 10/5

4/4 [==============================] - 0s 30ms/step - loss: 0.4559 - categorical_accuracy: 0.8596
Epoch 148/500
4/4 [==============================] - 0s 34ms/step - loss: 1.2492 - categorical_accuracy: 0.6140
Epoch 149/500
4/4 [==============================] - 0s 28ms/step - loss: 1.0809 - categorical_accuracy: 0.6842
Epoch 150/500
4/4 [==============================] - 0s 28ms/step - loss: 1.1310 - categorical_accuracy: 0.5702
Epoch 151/500
4/4 [==============================] - 0s 27ms/step - loss: 0.8964 - categorical_accuracy: 0.6667
Epoch 152/500
4/4 [==============================] - 0s 28ms/step - loss: 0.9086 - categorical_accuracy: 0.6228
Epoch 153/500
4/4 [==============================] - 0s 29ms/step - loss: 1.0229 - categorical_accuracy: 0.6754
Epoch 154/500
4/4 [==============================] - 0s 28ms/step - loss: 1.0475 - categorical_accuracy: 0.5000
Epoch 155/500
4/4 [==============================] - 0s 28ms/step - loss: 0.8822 - categorical_accuracy: 0.6754
Epoch 

4/4 [==============================] - 0s 32ms/step - loss: 0.2979 - categorical_accuracy: 0.8947
Epoch 294/500
4/4 [==============================] - 0s 34ms/step - loss: 0.2742 - categorical_accuracy: 0.9035
Epoch 295/500
4/4 [==============================] - 0s 30ms/step - loss: 3.0345 - categorical_accuracy: 0.6842
Epoch 296/500
4/4 [==============================] - 0s 30ms/step - loss: 3.3148 - categorical_accuracy: 0.3070
Epoch 297/500
4/4 [==============================] - 0s 29ms/step - loss: 1.2670 - categorical_accuracy: 0.4474
Epoch 298/500
4/4 [==============================] - 0s 30ms/step - loss: 1.2184 - categorical_accuracy: 0.4386
Epoch 299/500
4/4 [==============================] - 0s 30ms/step - loss: 1.2014 - categorical_accuracy: 0.3772
Epoch 300/500
4/4 [==============================] - 0s 32ms/step - loss: 1.2159 - categorical_accuracy: 0.3860
Epoch 301/500
4/4 [==============================] - 0s 32ms/step - loss: 1.2168 - categorical_accuracy: 0.4386
Epoch 

4/4 [==============================] - 0s 29ms/step - loss: 0.1441 - categorical_accuracy: 0.9386
Epoch 440/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1320 - categorical_accuracy: 0.9386
Epoch 441/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1269 - categorical_accuracy: 0.9298
Epoch 442/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1238 - categorical_accuracy: 0.9474
Epoch 443/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1675 - categorical_accuracy: 0.9298
Epoch 444/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1373 - categorical_accuracy: 0.9474
Epoch 445/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1594 - categorical_accuracy: 0.9474
Epoch 446/500
4/4 [==============================] - 0s 28ms/step - loss: 0.1362 - categorical_accuracy: 0.9386
Epoch 447/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1173 - categorical_accuracy: 0.9474
Epoch 

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 4)                 132       
                                                                 
Total params: 203,492
Trainable params: 203,492
Non-tr

# 8. Check Predictions

In [49]:
res = model.predict(x_test)

In [50]:
actions[np.argmax(res[4])]

'input'

In [51]:
actions[np.argmax(y_test[4])]

'input'

# 9. Save Model

In [52]:
model.save('model_rh_only.h5')

In [53]:
model.load_weights('model_rh_only.h5')

# 10. Evaluate Model Accuracy

In [54]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [55]:
yhat = model.predict(x_test)

In [56]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [57]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 2]],

       [[4, 0],
        [0, 2]],

       [[5, 0],
        [1, 0]],

       [[4, 1],
        [0, 1]]], dtype=int64)

In [58]:
accuracy_score(ytrue, yhat)

0.8333333333333334